# Análise de dados eleições Camaragibe 2024
Este projeto tem como objetivo otimizar o tratamento de dados de uma pesquisa eleitoral relacionada às eleições de Camaragibe em 2024. Anteriormente, o processamento dos dados era realizado manualmente, com a aplicação diária de filtros em diversas planilhas. O propósito deste projeto é automatizar o processo de filtragem e tratamento dos dados, garantindo uma análise mais eficiente e precisa das perguntas e respostas coletadas.

## Base de dados
A base de dados utilizada consiste em um arquivo no formato .xlsx (Excel), contendo cerca de 24 perguntas que compõem o questionário da pesquisa. Após o processamento, o sistema gera relatórios indicando a quantidade e a porcentagem de cada resposta relacionada aos blocos de perguntas analisados. Essas melhorias visam simplificar o fluxo de trabalho, reduzir erros manuais e fornecer insights rápidos e confiáveis sobre as respostas coletadas durante a pesquisa.

In [145]:
import os
import pandas as pd

nomeArquivo = "CAMARAGIBE - CÉU AZUL - 07.09.xlsx"
df = pd.read_excel(f"src/{nomeArquivo}", sheet_name="DADOS")

### 1. Sexo
Embora tenha comportamento RU Estimulada, deve-se sempre ter uma resposta válida (feminino/masculino), logo considera os itens vazios como a resposta com mior mostragem.

In [146]:
#1. Sexo
def sexo():
  # Tratamento geral das linhas vazias
  coluna = "1. SEXO"
  sexo = df[coluna].value_counts()
  valor_mais_frequente = sexo.idxmax()
  df[coluna] = df[coluna].fillna(valor_mais_frequente)

  contagem_sexo = df[coluna].value_counts()
  porcentagem_sexo = (df[coluna].value_counts(normalize = True) * 100).round(2)
  
  # Exibição em tabela
  resultado = pd.DataFrame({
      'Contagem': contagem_sexo,
      'Porcentagem': porcentagem_sexo.astype(str) + '%'
  })
    
  # Adicionar a linha do total
  total_contagem = contagem_sexo.sum()
  total_porcentagem = porcentagem_sexo.sum()

  if (total_porcentagem != 100.00):
    ajuste = 100.00 - total_porcentagem
    total_porcentagem += ajuste

  resultado.loc['Total'] = [total_contagem, f'{total_porcentagem:.2f}%']

  return resultado

### 2. Faixa etaria
Embora também apresente comportamento RU Estimulada, deve-se também sempre se manter com respostas válidas (de acordo com as faixas etárias válidas), logo, considera-se a faixa etária das linhas vazias como a faixa de maior mostragem.

In [147]:
def faixa_etaria():
  # Tratamento geral das linhas vazias
  coluna = "2. FAIXA ETÁRIA"
  faixa_etaria = df[coluna].value_counts()
  valor_mais_frequente = faixa_etaria.idxmax()
  df[coluna] = df[coluna].fillna(valor_mais_frequente)

  contagem_faixa_etaria = df[coluna].value_counts()
  porcentagem_faixa_etaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

  # Exibir Tabela
  resultado = pd.DataFrame({
      "contagem": contagem_faixa_etaria,
      "porcentagem": porcentagem_faixa_etaria.astype(str) + "%"
  })

  # Adiciona a linha do total
  total_contagem = contagem_faixa_etaria.sum()
  total_porcentagem = porcentagem_faixa_etaria.sum()

  if (total_porcentagem != 100.00):
    ajuste = 100.00 - total_porcentagem
    total_porcentagem += ajuste

  resultado.loc['Total'] = [total_contagem, f'{total_porcentagem:.2f}%']

  return resultado

### 3. Escolaridade
Pergunta referente a escolaridade do entrevistado. Respostas:


In [148]:
def escolaridade():
  # Tratamento geral das linhas vazias
  coluna = "3. ESCOLARIDADE"
  escolaridade = df[coluna].value_counts()
  valor_mais_frequente = escolaridade.idxmax()
  df[coluna] = df[coluna].fillna(valor_mais_frequente)

  contagem_escolaridade = df[coluna].value_counts()
  porcentagem_escolaridade = (df[coluna].value_counts(normalize = True) * 100).round(2)

  # Exibir Tabela
  resultado = pd.DataFrame({
      "contagem": contagem_escolaridade,
      "porcentagem": porcentagem_escolaridade.astype(str) + "%"
  })

  # Adiciona a linha do total
  total_contagem = contagem_escolaridade.sum()
  total_porcentagem = porcentagem_escolaridade.sum()
    
  if (total_porcentagem != 100.00):
    ajuste = 100.00 - total_porcentagem
    total_porcentagem += ajuste

  resultado.loc['Total'] = [total_contagem, f'{total_porcentagem:.2f}%']

  return resultado

### Tratamento perguntas espontâneas

In [149]:
def tratamento_ru_espontanea(coluna):
  substituicoes = {
        "DIEGO CABRAL": ["diego", "cabral", "diego cabral", "diego alves"],
        "JORGE ALEXANDRE": ["jorge", "jorge alexandre", "alexandre", "jorge ale", "jorge alixandre", "jorge vieira", "ale xandre"],
        "FELIPE DANTAS": ["felipe", "dantas", "felipe dantas"],
        "BOSCO": ["bosco"],
        "NENHUM": ["não", "nulo", "n", "nenhum", "nao sabe", "n sei", "n quero opinar", "n quero responder", "nada a declarar", "n faz ideia", "n opinou"],
        "NÃO SABE": ["não sabe", "nao opnou", "nao opinou", "ñ sei", "indecisa", "indeciso", ]
    }
  
  # Tratamento geral dos espaços vazios e repetidos
  df[coluna] = df[coluna].fillna("NENHUM")

  # Aplicando a substituição
  for candidato in df[coluna]:
      if (candidato.lower().strip() in substituicoes["DIEGO CABRAL"]):
          df[coluna] = df[coluna].replace(candidato, "DIEGO CABRAL")

      elif (candidato.lower().strip() in substituicoes["JORGE ALEXANDRE"]):
          df[coluna] = df[coluna].replace(candidato, "JORGE ALEXANDRE")

      elif (candidato.lower().strip() in substituicoes["FELIPE DANTAS"]):
          df[coluna] = df[coluna].replace(candidato, "FELIPE DANTAS")

      elif (candidato.lower().strip() in substituicoes["BOSCO"]):
          df[coluna] = df[coluna].replace(candidato, "BOSCO")

      elif (candidato.lower().strip() in substituicoes["NENHUM"]):
          df[coluna] = df[coluna].replace(candidato, "NENHUM")

      elif (candidato.lower().strip() in substituicoes["NÃO SABE"]):
          df[coluna] = df[coluna].replace(candidato, "NÃO SABE/NÃO RESPONDEU")
        
  contagem_votaria = df[coluna].value_counts()
  porcentagem_votaria = (df[coluna].value_counts(normalize = True) * 100).round(2)

  # Exibição da tabela
  resultado = pd.DataFrame({
      "contagem": contagem_votaria,
      "porcentagem": porcentagem_votaria.astype(str) + "%"
  })

  # Total
  total_contagem = contagem_votaria.sum()
  total_porcentagem = porcentagem_votaria.sum()
    
  if (total_porcentagem != 100.00):
      ajuste = 100.00 - total_porcentagem
      total_porcentagem += ajuste

  resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]

  return resultado

perguntas_espontanea = [
    "5. SE AS ELEIÇÕES PARA PREFEITO DE CAMARAGIBE FOSSE HOJE, EM QUAL CANDIDATO VOCÊ VOTARIA? (RU ESPONTÂNEA)",
    "17. A PREFEITA NADEGI NÃO PODERÁ SER MAIS CANDIDATA NA PRÓXIMA ELEIÇÃO. QUEM É O CANDIDATO A PREFEITO QUE ELA IRÁ APOIAR? (RU ESPONTÂNEA)"
]

### Tratamento de dados das perguntas RU Estimulada

In [150]:
def tratamento_ru_estimulada(coluna):
  # Tratamento geral dos espaços vazios e repetidos
    df[coluna] = df[coluna].fillna("NÃO SABE/NÃO RESPONDEU")

    contagem = df[coluna].value_counts()
    porcentagem = (df[coluna].value_counts(normalize = True) * 100).round(2)

    # Exibição da tabela
    resultado = pd.DataFrame({
        "contagem": contagem,
        "porcentagem": porcentagem.astype(str) + "%"
    })

    # Total
    total_contagem = contagem.sum()
    total_porcentagem = porcentagem.sum()
    
    if (total_porcentagem != 100.00):
        ajuste = 100.00 - total_porcentagem
        total_porcentagem += ajuste

    resultado.loc["Total"] = [total_contagem, f"{total_porcentagem:.2f}%"]

    return resultado

perguntas_estimulada = [
    "4. COMO VOCÊ CLASSIFICARIA O SEU INTERESSE NAS ELEIÇÕES MUNICIPAIS DESTE ANO? (RU ESTIMULADA)",
    "6. SE AS ELEIÇÕES PARA PREFEITO DE CAMARAGIBE FOSSEM HOJE, EM QUAIS DESSES CANDIDATOS VOCÊ VOTARIA? (RU ESTIMULADA)",
    "7. INDEPENDENTE DA SUA INTENÇÃO DE VOTOS, QUEM VOCÊ ACREDITA QUE ESTEJA MAIS PREPARADO PARA SER PREFEITO DE CAMARAGIBE? (RU ESTIMULADA)",
    "8. INDEPENDENTE DA SUA INTENÇÃO DE VOTOS, QUEM VOCÊ ACREDITA QUE GANHARÁ A ELEIÇÃO PARA PREFEITO DE CAMARAGIBE? (RU ESTIMULADA)",
    "9. EM RELAÇÃO A SUA INTENÇÃO DE VOTO PARA PREFEITO DE CAMARAGIBE, HOJE VOCÊ DIRIA QUE: (RU ESTIMULADA)",
    "10. SOBRE A CANDIDATURA A PREFEITO DE CAMARAGIBE DE DIEGO CABRAL, VOCÊ DIRIA QUE: (RU ESTIMULADA)",
    "11. (EXCETO PARA QUEM NÃO CONHECE) E VOCÊ POSSUI UMA IMAGEM POSITIVA OU NEGATIVA DO PRÉ-CANDIDATO A PREFEITO DE CAMARAGIBE DIEGO CABRAL: (RU ESTIMULADA)",
    "12. SOBRE A CANDIDATURA A PREFEITO DE CAMARAGIBE DE JORGE ALEXANDRE, VOCÊ DIRIA QUE: (RU ESTIMULADA)",
    "13. (EXCETO PARA QUEM NÃO O CONHECE) E VOCÊ POSSUI UMA IMAGEM POSITIVA OU NEGATIVA DO PRÉ-CANDIDATO A PREFEITO DE CAMARAGIBE JORGE ALEXANDRE: (RU ESTIMULADA)",
    "14. SOBRE A CANDIDATURA A PREFEITO DE CAMARAGIBE DE BOSCO, VOCÊ DIRIA QUE: (RU ESTIMULADA)",
    "15. (EXCETO PARA QUEM NÃO O CONHECE) E VOCÊ POSSUI UMA IMAGEM POSITIVA OU NEGATIVA DO PRÉ-CANDIDATO A PREFEITO DE CAMARAGIBE BOSCO: (RU ESTIMULADA)",
    "16. SE AS ELEIÇÕES PARA PREFEITO DE CAMARAGIBE FOSSEM HOJE, EM QUEM VOCÊ VOTARIA? (RU ESTIMULADA)",
    "18. SE AS ELEIÇÕES PARA PREFEITO DE CAMARAGIBE FOSSEM HOJE, VOCÊ VOTARIA EM DIEGO CABRAL SE O PRESIDENTE LULA APOIAR? (RU ESTIMULADA)",
    "19. DE MANEIRA GERAL, COMO VOCÊ AVALIA A GESTÃO DA GOVERNADORA RAQUEL LYRA? (RU ESTIMULADA)",
    "20. SOBRE A GESTÃO DA PREFEITA NADEGI, COMO O (A) SR. (A) AVALIA A SUA ADMINISTRAÇÃO A FRENTE DA PREFEITURA DE CAMARAGIBE? (RU ESTIMULADA)",
    "21. DE MANEIRA GERAL, SOBRE A FORMA DE ADMINISTRAR DA PREFEITA NADEGI, O (A) SR. (A) APROVA OU DESAPROVA? (RU ESTIMULADA)",
    "22. EM QUAL DESSES CANDIDATOS VOCÊ NÃO VOTARIA DE MANEIRA ALGUMA? (RU ESTIMULADA)",
    "23. (NÃO PEA) DAS ALTERNATIVAS QUE VOU LER, QUAL MELHOR REPRESENTA A SUA SITUAÇÃO EM RELAÇÃO AO TRABALHO?",
    "24. RENDA (PESSOAL)",
    "25. RELIGIÃO"
    ]

# Exportando análise

In [151]:
# Criar um novo arquivo Excel com os resultados
with pd.ExcelWriter(f"results/{nomeArquivo}", engine="openpyxl") as writer:
    # Aplicando o tratamento para perguntas específicas
    sexo = sexo()
    sexo.to_excel(writer, sheet_name=f"Espontanea_{"1. SEXO"}", index=True)

    faixa_etaria = faixa_etaria()
    faixa_etaria.to_excel(writer, sheet_name=f"Espontanea_{"2. FAIXA ETÁRIA"}", index=True)

    escolaridade = escolaridade()
    escolaridade.to_excel(writer, sheet_name=f"Espontanea_{"3. ESCOLARIDADE"}", index=True)
    
    # Aplicar o tratamento para RU Espontânea e salvar os resultados
    for coluna in perguntas_espontanea:
        resultado = tratamento_ru_espontanea(coluna)
        resultado.to_excel(writer, sheet_name=f"Espontanea_{coluna[:10]}", index=True)
        
    # Aplicar o tratamento para RU Estimulada e salvar os resultados
    for coluna in perguntas_estimulada:
        resultado = tratamento_ru_estimulada(coluna)
        resultado.to_excel(writer, sheet_name=f"Estimulada_{coluna[:10]}", index=True)

print("Processo concluído. Os resultados foram salvos em 'resultados_tratamento.xlsx'.")

Processo concluído. Os resultados foram salvos em 'resultados_tratamento.xlsx'.
